<a href="https://colab.research.google.com/github/kenji114/GoogleColabByHand/blob/master/CUDA_GPU_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Created by Imad El Hanafi : https://imadelhanafi.com

# Check GPU status

Make surre to use : GPU runtime mode (Runtime->Change Runtime type -> python3 + GPU
)



In [0]:
# Check nvidia and nvcc cuda compiler

!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Tue Dec 24 12:50:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#Mount Goolge Drive

In [0]:
# link to google drive

from google.colab import drive
drive.mount('/content/gdrive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
#check that Gdrive is mounted

!ls '/content/gdrive/My Drive/Colab Notebooks/ssh_files/'

test.gdoc  Untitled.ipynb


#Setup SSH port forwarding

In [0]:
#1 - setup ssh/user 


#Generate a random root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))


#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", password)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

username: root
password:  yG8gwhyr3m9DBZD5YS7nBOGMYWpgbT


In [0]:
# 2 - Download Ngrok

! rm -rf ngrok-stable-linux-amd64.zip*
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [0]:
# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

! rm -rf /root/.ngrok2/
! mkdir -p /root/.ngrok2/
! echo "authtoken:" $authtoken >> /root/.ngrok2/ngrok.yml
! echo "tunnels:" >> /root/.ngrok2/ngrok.yml
! echo "  ssh:" >> /root/.ngrok2/ngrok.yml
! echo "    addr: 22" >> /root/.ngrok2/ngrok.yml
! echo "    proto: tcp" >> /root/.ngrok2/ngrok.yml
! echo "  jupyter:" >> /root/.ngrok2/ngrok.yml
! echo "    addr: 8888" >> /root/.ngrok2/ngrok.yml
! echo "    proto: http" >> /root/.ngrok2/ngrok.yml

#Create tunnel
get_ipython().system_raw('./ngrok start ssh jupyter &')
# get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

Get your authtoken from https://dashboard.ngrok.com/auth
··········


In [0]:
! apt-get install tmux
! pip install jupyterlab

In [0]:
! tmux new-sess -d 'jupyter lab --ip 0.0.0.0 --no-browser --allow-root '

Congratulations, you are ready to go. 
On Ngrok interface https://dashboard.ngrok.com/status you'll find the tcp address and the port 

connect using the following : 

```
ssh root@0.tcp.ngrok.io -p [ngrok_port]

> then enter the password generated previously

```

In [0]:
!kill $(ps aux | grep 'jupyter' | awk '{print $2}')

In [0]:
# When done, kill Ngrok

!kill $(ps aux | grep './ngrok' | awk '{print $2}')

^C
